In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow

In [ ]:
!pip install git+https://github.com/MarcoGorelli/narwhals.git@more-dask-tpch

In [ ]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [ ]:
from typing import Any
import narwhals as nw

@nw.narwhalify
def q2(
    region_ds: Any,
    nation_ds: Any,
    supplier_ds: Any,
    part_ds: Any,
    part_supp_ds: Any,
) -> Any:
    var_1 = 15
    var_2 = "BRASS"
    var_3 = "EUROPE"

    result_q2 = (
        part_ds.join(part_supp_ds, left_on="p_partkey", right_on="ps_partkey")
        .join(supplier_ds, left_on="ps_suppkey", right_on="s_suppkey")
        .join(nation_ds, left_on="s_nationkey", right_on="n_nationkey")
        .join(region_ds, left_on="n_regionkey", right_on="r_regionkey")
        .filter(
            nw.col("p_size") == var_1,
            nw.col("p_type").str.ends_with(var_2),
            nw.col("r_name") == var_3,
        )
    )

    final_cols = [
        "s_acctbal",
        "s_name",
        "n_name",
        "p_partkey",
        "p_mfgr",
        "s_address",
        "s_phone",
        "s_comment",
    ]

    return (
        result_q2.group_by("p_partkey")
        .agg(nw.col("ps_supplycost").min().alias("ps_supplycost"))
        .join(
            result_q2,
            left_on=["p_partkey", "ps_supplycost"],
            right_on=["p_partkey", "ps_supplycost"],
        )
        .select(final_cols)
        .sort(
            ["s_acctbal", "n_name", "s_name", "p_partkey"],
            descending=[True, False, False, False],
        )
        .head(100)
    )

In [ ]:
# dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
dir_ = "~/tpch-data/s2/"
region = dir_ + 'region.parquet'
nation = dir_ + 'nation.parquet'
customer = dir_ + 'customer.parquet'
lineitem = dir_ + 'lineitem.parquet'
orders = dir_ + 'orders.parquet'
supplier = dir_ + 'supplier.parquet'
part = dir_ + 'part.parquet'
partsupp = dir_ + 'partsupp.parquet'

In [ ]:
import pyarrow.parquet as pq
import dask.dataframe as dd

IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
    'pyarrow': lambda x: pq.read_table(x),
    'dask': lambda x: dd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
}

In [ ]:
results = {}

## pandas

In [ ]:
tool = 'pandas'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp))
results[tool] = timings.all_runs

## pandas, pyarrow dtypes

In [ ]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp))
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp))
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp)).collect()
results[tool] = timings.all_runs

## PyArrow

In [ ]:
tool = 'pyarrow'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp))
results[tool] = timings.all_runs

## Dask

In [ ]:
tool = 'dask'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q2(fn(region), fn(nation), fn(supplier), fn(part), fn(partsupp)).compute()
results[tool] = timings.all_runs

## Save

In [ ]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)
